In [1]:
import os

In [2]:
%pwd

'/Users/shifaafreensiddiqui/Desktop/Projects/NLP/Text-Summarization/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/shifaafreensiddiqui/Desktop/Projects/NLP/Text-Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [11]:
import os
import json
from datasets import Dataset
from transformers import AutoTokenizer
from textSummarizer.logging import logger


In [ ]:
# class DataTransformation:
#     def __init__(self, config: DataTransformationConfig):
#         self.config = config
#         self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


#     def convert_examples_to_features(self, example_batch):
#         inputs = self.tokenizer(
#             example_batch['dialogue'], 
#             max_length=1024, 
#             truncation=True, 
#             padding="max_length"
#         )
        
#         targets = self.tokenizer(
#             text_target=example_batch['summary'], 
#             max_length=128, 
#             truncation=True, 
#             padding="max_length"
#         )
        
#         return {
#             'input_ids': inputs['input_ids'],
#             'attention_mask': inputs['attention_mask'],
#             'labels': targets['input_ids']
#         }

#     def convert(self):
#         splits = ["train", "validation", "test"]

#         for split in splits:
#             path = os.path.join(self.config.data_path, f"{split}.json")
#             logger.info(f"Loading {split} data from {path}")
            
#             with open(path, "r") as f:
#                 raw_data = [json.loads(line) for line in f.readlines()]
            
#             dataset = Dataset.from_list(raw_data)
#             transformed_dataset = dataset.map(self.convert_examples_to_features, batched=True)

#             save_path = os.path.join(self.config.root_dir, f"{split}_transformed")
#             transformed_dataset.save_to_disk(save_path)
#             logger.info(f"Saved transformed {split} data to {save_path}")


import os
import json
from datasets import Dataset
from transformers import AutoTokenizer
from textSummarizer.logging import logger


class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    # def convert_examples_to_features(self, example_batch):
    #     input_encodings = self.tokenizer(
    #         example_batch['dialogue'],
    #         max_length=1024,
    #         truncation=True
    #     )

    #     target_encodings = self.tokenizer(
    #         example_batch['summary'],
    #         max_length=128,
    #         truncation=True,
    #         text_target=example_batch['summary']
    #     )

    #     return {
    #         'input_ids': input_encodings['input_ids'],
    #         'attention_mask': input_encodings['attention_mask'],
    #         'labels': target_encodings['input_ids']
    #     }

    # def convert(self):
    #     for split in ['train', 'test', 'validation']:
    #         path = os.path.join("artifacts", "data_ingestion", f"{split}.json")
    #         logger.info(f"Loading {split} data from {path}")

    #         with open(path, "r") as f:
    #             raw_data = [json.loads(line) for line in f.readlines()]

    #         dataset = Dataset.from_list(raw_data)
    #         transformed_dataset = dataset.map(self.convert_examples_to_features, batched=True)

    #         save_path = os.path.join(self.config.root_dir, f"{split}_transformed")
    #         transformed_dataset.save_to_disk(save_path)




    def convert_examples_to_features(self, example_batch):
        dialogues = example_batch.get("dialogue", [])
        summaries = example_batch.get("summary", [])

        for d, s in zip(dialogues, summaries):
            if not isinstance(d, str) or not isinstance(s, str):
                print("Invalid input:", d, "|", s)

        # Filter out non-string values
        def clean(texts):
            return [
                str(t) if isinstance(t, (str, int, float)) else ""
                for t in texts
            ]

        dialogues = clean(dialogues)
        summaries = clean(summaries)

        inputs = self.tokenizer(
            dialogues,
            max_length=1024,
            truncation=True,
            padding="max_length"
        )

        targets = self.tokenizer(
            summaries,
            max_length=128,
            truncation=True,
            padding="max_length"
        )

        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": targets["input_ids"]
        }

    



    def convert(self):
        for split in ['train', 'test', 'validation']:
            path = os.path.join("artifacts", "data_ingestion", f"{split}.json")
            logger.info(f"Loading {split} data from {path}")

            with open(path, "r") as f:
                raw_data = [json.loads(line) for line in f.readlines()]

            dataset = Dataset.from_list(raw_data)
            transformed_dataset = dataset.map(self.convert_examples_to_features, batched=True)

            save_path = os.path.join(self.config.root_dir, f"{split}_transformed")
            transformed_dataset.save_to_disk(save_path)
    

In [25]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-06-08 11:43:28,025: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-08 11:43:28,026: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-08 11:43:28,027: INFO: common: created directory at: artifacts]
[2025-06-08 11:43:28,027: INFO: common: created directory at: artifacts/data_transformation]
[2025-06-08 11:43:29,526: INFO: 1176555295: Loading train data from artifacts/data_ingestion/train.json]


Map:  48%|████▊     | 7000/14732 [00:01<00:01, 5539.43 examples/s]

Invalid input: None | problem with visualization of the content


Saving the dataset (1/1 shards): 100%|██████████| 14732/14732 [00:00<00:00, 417074.94 examples/s]

[2025-06-08 11:43:32,402: INFO: 1176555295: Loading test data from artifacts/data_ingestion/test.json]



Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 306489.56 examples/s]

[2025-06-08 11:43:32,585: INFO: 1176555295: Loading validation data from artifacts/data_ingestion/validation.json]



Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 311281.14 examples/s]
